In [ ]:
from datasets import load_dataset
from random import randrange
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer
import mlflow

In [ ]:
model_name = "Salesforce/codegen-350M-multi"
dataset_name = "Shreyasrp/Text-to-SQL"
split = "train[:10%]"
finetuned_model_name = "ft-codegen-350M"
device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device_map)

In [ ]:
# BitsAndBytesConfig config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

peft_config = LoraConfig(
      r=16,
      lora_alpha=32,
      lora_dropout=0.05,
      bias="none",
      task_type="CAUSAL_LM",
)

def prompt_instruction_format(sample):
  return f"""### Task:
    {sample['instruction']}

    ### Response:
    {sample['output']}
    """

In [ ]:
dataset = load_dataset(dataset_name, split=split)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, use_cache = False, device_map=device_map)
base_model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"
tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = True

In [ ]:
# Define the training arguments
trainingArgs = TrainingArguments(
    output_dir=finetuned_model_name, # Output directory where the model predictions and checkpoints will be stored
    num_train_epochs=3, # Number of training epochs
    #per_device_train_batch_size=4, # Batch size per GPU for training
    auto_find_batch_size=True,
    gradient_accumulation_steps=2,  # Number of update steps to accumulate the gradients for
    gradient_checkpointing=True,# Enable gradient checkpointing
    optim="paged_adamw_32bit", # Optimizer to use
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    #weight_decay=0.001,
    fp16=False,
    bf16=False,
    #max_grad_norm=0.3,
    #warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    seed=42
)
# Create the trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

In [ ]:
print("Start Supervised Fine-Tuning")
mlflow.set_experiment("Fine-Tuning codeGen-350M")
trainer.train()
print("Training Done")

# save model
trainer.save_model()
print("Model Saved")

In [ ]:
# load the trained model from the output directory
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    trainingArgs.output_dir,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# Merge LoRA with the base model and save the merged model
lora_merged_model = trained_model.merge_and_unload()
lora_merged_model.save_pretrained("merged",safe_serialization=True)
tokenizer.save_pretrained("merged")

In [ ]:
mytask="CREATE TABLE head (age INTEGER), create sql to query How many heads of the departments are older than 56."

prompt = f"""
# Instruction:
Use the Task below to write the Response.
# Task:
{mytask}
# Response:
"""

#input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).to(device_map)
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device_map)
attention_mask = torch.ones(input_ids.shape, dtype=torch.long).to(device_map)
# with torch.inference_mode():

print(f"-------------------------\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n")

print(f"Base Model Response :\n")
output_base = base_model.generate(input_ids=input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5,attention_mask=attention_mask)
print(f"{tokenizer.batch_decode(output_base.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n")

print(f"LORA Model Response :\n")
output_trained_lora = lora_merged_model.generate(input_ids=input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5,attention_mask=attention_mask)
print(f"{tokenizer.batch_decode(output_trained_lora.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n")

In [ ]:
import gc

# clear the VRAM
del base_model
del trained_model
del lora_merged_model
del trainer
torch.cuda.empty_cache()
gc.collect()

In [ ]:
from datasets import load_dataset
from random import randrange
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer

device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device_map)

loaded_models = {}
mymodel1 = "merged"
tokenizer = AutoTokenizer.from_pretrained(mymodel1,torch_dtype=torch.bfloat16,device_map="auto")
model = AutoModelForCausalLM.from_pretrained(mymodel1,torch_dtype=torch.bfloat16,device_map="auto")
loaded_models[mymodel1] = {"tokenizer": tokenizer, "model": model}
tokenizer1 = loaded_models[mymodel1]["tokenizer"]
base_model = loaded_models[mymodel1]["model"]

loaded_models = {}
mymodel2 = "codegen-350M-multi"
tokenizer = AutoTokenizer.from_pretrained(mymodel2,torch_dtype=torch.bfloat16,device_map="auto")
model = AutoModelForCausalLM.from_pretrained(mymodel2,torch_dtype=torch.bfloat16,device_map="auto")
loaded_models[mymodel2] = {"tokenizer": tokenizer, "model": model}
tokenizer2 = loaded_models[mymodel2]["tokenizer"]
lora_merged_model = loaded_models[mymodel2]["model"]

In [ ]:
mytask="CREATE TABLE head (age INTEGER), create sql to query How many heads of the departments are older than 56."
prompt = f"""
# Instruction:
Use the Task below to write the Response.
# Task:
{mytask}
# Response:
"""

#input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).to(device_map)
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device_map)
#input_ids = tokenizer.encode(mytask, return_tensors="pt").to(device_map)
attention_mask = torch.ones(input_ids.shape, dtype=torch.long).to(device_map)
# with torch.inference_mode():

print(f"-------------------------\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n")

print(f"merged Model Response :\n")
output_base = base_model.generate(input_ids=input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5,attention_mask=attention_mask)
print(f"{tokenizer1.batch_decode(output_base.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n")

print(f"codegen-350m Model Response :\n")
output_trained_lora = lora_merged_model.generate(input_ids=input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5,attention_mask=attention_mask)
print(f"{tokenizer2.batch_decode(output_trained_lora.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n")